Един възможен подход за ефективно пресмятане на стойност на функция в точка е т.нар. lookup table
    (вж.https://en.wikipedia.org/wiki/Lookup_table).

Да се напише фунцкия lookup_table_f(x_val), която пресмята приближено стойността на функцията 𝑓(𝑥)=sin(sqrt(x))
    в интервала [0,2𝜋], като използва 2 предварително генерирани масива - f_nodes,
        съдържащ равноотдалечени на разстояние ℎ=0.01 точки в интервала [0,2𝜋] и f_values - масив от стойности на фунцкията f,
            пресметнати в точките f_nodes.
Функцията трябва да пресмята приближено стойността на 𝑓(𝑥)
    като използва кубична интерполация в най-близките до x_val възли от списъка f_nodes. За стойности на x_val,
        които принадлежат на интервалите [0,0.01] и [2𝜋−0.01,2𝜋] да се построява интерполационен полином от 𝜋2.
В случай, че подаденият аргумент x_val лежи извън интервала [0,2𝜋], да се извежда подходящо съобщение.

In [158]:
import numpy as np
import bisect

In [159]:
import sys
sys.path.append('../util')

from lagrange_poly import lagrange_poly

In [160]:
def look_up_table_f(x, function=lambda x: np.sin(np.sqrt(x)), interval_begin=0, interval_end=2*np.pi, h=0.01, verbose=False):
    if x < interval_begin or x > interval_end:
        return None
    
    interval_begin = round(interval_begin, 2)
    interval_end = round(interval_end, 2)

    nodes = [i * h for i in range(int((interval_end - interval_begin) * 100) + 1)]
    values = function(nodes)

    def get_neighbors(x):
        for i in range(len(nodes) - 1):
            if nodes[i] == x:
                return nodes[i-1], nodes[i+1]
            if nodes[i] < x and x < nodes[i+1]:
                return nodes[i], nodes[i+1]

    get_index = lambda node_value: bisect.bisect_left(nodes, node_value)

    def interpolate(nodes, values, x):
        if verbose == True:
            print('Interpolating the nodes:', nodes, 'for x:', x)

        return lagrange_poly(nodes, values, x)

    if x - interval_begin <= h:
        interpolation_nodes = np.array(nodes[0:3])
        interpolation_values = np.array([
            values[0],
            values[1],
            values[2],
        ])
        
        return interpolate(interpolation_nodes, interpolation_values, x)

    if interval_end - x <= h:
        nodes = np.flip(nodes)
        values = np.flip(values)

        interpolation_nodes = np.array(nodes[0:3])
        interpolation_values = np.array([
            values[0],
            values[1],
            values[2],
        ])

        interpolation_nodes = np.flip(interpolation_nodes)
        interpolation_values = np.flip(interpolation_values)

        return interpolate(interpolation_nodes, interpolation_values, x)

    left1, right1 = get_neighbors(x)

    left2, _ = get_neighbors(left1)
    _, right2 = get_neighbors(right1)

    interpolation_nodes = np.array([
        left2,
        left1,
        right1,
        right2,
    ])

    interpolation_values = np.array([
        values[get_index(left2)],
        values[get_index(left1)],
        values[get_index(right1)],
        values[get_index(right2)],
    ])

    return interpolate(interpolation_nodes, interpolation_values, x)

In [161]:
print('Approximated value of sin(sqrt(1.602311))', look_up_table_f(1.602311, verbose=True))
print('Exact value of sin(sqrt(1.602311))', np.sin(np.sqrt(1.602311)))

Interpolating the nodes: [1.59 1.6  1.61 1.62] for x: 1.602311
Approximated value of sin(sqrt(1.602311)) 0.9538553335959832
Exact value of sin(sqrt(1.602311)) 0.9538553335617003


In [162]:
x = 0.0075

print(f'Approximated value of sin(sqrt({x}))', look_up_table_f(x, verbose=True))
print(f'Exact value of sin(sqrt({x}))', np.sin(np.sqrt(x)))

Interpolating the nodes: [0.   0.01 0.02] for x: 0.0075
Approximated value of sin(sqrt(0.0075)) 0.08037972595984295
Exact value of sin(sqrt(0.0075)) 0.08649432779066327


In [163]:
print('Approximated value of sin(sqrt(6.28))', look_up_table_f(6.28, verbose=True))
print('Exact value of sin(sqrt(6.28))', np.sin(np.sqrt(6.28)))

Interpolating the nodes: [6.26 6.27 6.28] for x: 6.28
Approximated value of sin(sqrt(6.28)) 0.5936603188902636
Exact value of sin(sqrt(6.28)) 0.5936603188902636
